# Generación de la Base de Datos y Monitorización del Incremento de SNPs:

1. Generación de paquetes de 500 muestras distribuidas aleatoriamente.

2. Filtrado de SNPs y Construcción de la Base de Datos.

3. Generación archivo curva de Rarefacción.

In [ ]:
import pandas as pd
import os
import glob
import random
from datetime import datetime

# Fijar semilla de números aleatorios
random.seed(0)

In [ ]:
input_directory = "/home/laura/andrea/scripts/TB_DDBB_Building/Paquetes_500_muestras/"
input_directory2 = "/media/NASII/Datos/ANALYSIS/Proyectos/BBDD_TB/AncestorII/Illumina/Variants"
samples_directory = "/home/laura/andrea/scripts/TB_DDBB_Building/Muestras"

pack_number = 1

df_files = pd.DataFrame(columns=['POS', 'REF', 'ALT', 'COUNT', 'SAMPLES'])

SNP_monitoring_df = pd.DataFrame(columns=['PACK', 'num_SNPs'])

################ Creación de enlaces simbólicos del paquete de muestras a procesar ################

# Obtener nombres del total de muestras analizadas del directorio de almacenamiento
folder_names_set = set(os.listdir(input_directory2))

# Iterar sobre los archivos indicados en los paquetes que se encuentran en input_directory
for pack_file in sorted(os.listdir(input_directory)):

    pack_path = os.path.join(input_directory, pack_file)
    
    # Obtener nombres de las muestras del paquete
    with open(pack_path, "r") as f:
        samples_pack = set(f.read().splitlines())

    # Si los nombres de las muestras del paquete (n = 500) se encuentran en la lista de nombres del directorio de almacenamiento (n = 100mil)
    for sample_name in samples_pack:
        if sample_name in folder_names_set:

            # Se busca el archivo tsv en el directorio con el nombre de la muestra
            sample_folder = os.path.join(input_directory2, sample_name)
            tsv_files = sorted(glob.glob(os.path.join(sample_folder, "snps.all.ivar.tsv")))

            for tsv_file in tsv_files:

                tsv_file_name = f'{sample_name}_snps.all.ivar.tsv'
                link_name = os.path.join(samples_directory, tsv_file_name)

                # Se crea enlace simbólico a ese archivo tsv en el directorio de salida
                if not os.path.exists(link_name): # Evitar errores si ya existe el enlace
                    os.symlink(tsv_file, link_name)


    print(f"Los enlaces simbólicos a los archivos de las muestras del paquete {pack_number} ya están disponibles en {samples_directory}.")

    ################ Ejecución del procesamiento de SNPs ################

    # Búsqueda de archivos TSV en todas las subcarpetas:
    for folder_path, subfolders, files in os.walk(samples_directory):

        subfolders.sort()
        files.sort()

        if os.path.isdir(folder_path):
            tsv_files = sorted(glob.glob(os.path.join(folder_path, "*_snps.all.ivar.tsv")))  # Busca archivos en cada subcarpeta con el patrón especificado

            for file in tsv_files:
                try:
                    df_file_sample = pd.read_csv(file, sep='\t')

                    sample_name = os.path.basename(file).split("_")[0] # Se queda con el ID de la muestra
                    df_file_sample["SAMPLES"] = sample_name

                    # Filtro de calidad
                    df_file_sample = df_file_sample[(df_file_sample['TOTAL_DP'] >= 20) & (df_file_sample['ALT_FREQ'] >= 0.7)]
                            
                    # Conserva solo los SNPs (se eliminan INDELs)
                    df_file_sample = df_file_sample[(df_file_sample['TYPE'] == "snp")]

                    # Se eliminan aquellos SNPS que caen en regiones complejas (muchos SNPs marcadores dentro de un intervalo de distancia pequeño)
                    df_file_sample = df_file_sample[(df_file_sample['OLDVAR'] != "complex")]

                    if not df_file_sample.empty:

                        # Solo para el primer paquete
                        if df_files.empty:
                            df_files = df_file_sample

                        # Para el resto de paquetes
                        else:
                            df_files = pd.concat([df_files, df_file_sample], ignore_index = True)
                
                except Exception as e:
                    print(f'Error procesando {file}: {e}')

    # Fusionar filas con el mismo POS, REF y ALT:
    df_files = df_files.groupby(['POS', 'REF', 'ALT'], as_index=False).agg({'SAMPLES': lambda x: ', '.join(sorted(set(x)))}) # Unir nombres de muestras sin duplicados
        
    df_files['COUNT'] = df_files['SAMPLES'].apply(lambda x: len(x.split(", ")))

    print(df_files)

    ################ Generación del archivo registro de la evolución de la BBDD ################

    new_row = pd.DataFrame({'PACK': [pack_number], 'num_SNPs': [len(df_files)]})
    SNP_monitoring_df = pd.concat([SNP_monitoring_df, new_row], ignore_index=True)

    ################ Eliminación de enlaces simbólicos del paquete anterior ################

    files_to_remove = glob.glob(os.path.join(samples_directory, "*_snps.all.ivar.tsv"))

    for file in files_to_remove:
        if os.path.islink(file):
            os.unlink(file)

        # Verificación extra de que se han eliminado los enlaces simbólicos
        elif os.path.exists(file):
            os.remove(file)

    print (f"Los enlaces simbólicos del paquete {pack_number} han sido eliminados")

    ################ Nuevo paquete de 500 muestras ################

    pack_number += 1
    print ("Inicio del procesamiento del siguiente paquete de muestras")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_df_files = f'/home/laura/andrea/scripts/TB_DDBB_Building/Filtrado_SNPs/Filtered_SNPs_{timestamp}.tsv'
df_files.to_csv(output_df_files, sep='\t', index=False)

output_SNP_monitoring_df = f'/home/laura/andrea/scripts/TB_DDBB_Building/Curva_Rarefaccion/SNP_monitoring_curve_{timestamp}.tsv'
SNP_monitoring_df.to_csv(output_SNP_monitoring_df, sep='\t', index=False)